In [1]:
import time

import ray
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.hyperopt import HyperOptSearch
from hyperopt import hp

In [2]:
def evaluate(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100) ** (-1) + height * 0.1

In [3]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"])
        session.report({"iterations": step, "mean_loss": score})

In [4]:
initial_params = [
    {"width": 1, "height": 2, "activation": "relu"},
    {"width": 4, "height": 2, "activation": "tanh"},
]
algo = HyperOptSearch(points_to_evaluate=initial_params)
algo = ConcurrencyLimiter(algo, max_concurrent=4)

In [5]:
num_samples = 100

In [6]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"])
}

In [7]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

2022-10-18 15:21:24,807	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-10-18 15:21:25,588	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_dbda6994,TERMINATED,127.0.0.1:29547,relu,2,100,1,1.11743,100,10.7082,99,-1.11743
objective_dcb5d6fa,TERMINATED,127.0.0.1:29548,tanh,2,100,4,0.446305,100,10.6878,99,-0.446305
objective_dcb75f0c,TERMINATED,127.0.0.1:29551,tanh,77.8997,100,7.53467,7.92225,100,10.6582,99,-7.92225
objective_dccb7000,TERMINATED,127.0.0.1:29552,relu,-79.9638,100,6.31132,-7.83886,100,10.6608,99,7.83886
objective_e319a544,TERMINATED,127.0.0.1:29569,tanh,10.0084,100,1.7202,1.55547,100,10.6542,99,-1.55547
objective_e3dd0c50,TERMINATED,127.0.0.1:29570,relu,39.7846,100,2.55452,4.35883,100,10.6106,99,-4.35883
objective_e3ed3206,TERMINATED,127.0.0.1:29552,relu,-81.8782,100,6.12313,-8.02553,100,10.6615,99,8.02553
objective_e3eeb7e8,TERMINATED,127.0.0.1:29571,tanh,-39.0726,100,19.5045,-3.85574,100,12.0038,99,3.85574
objective_ea2c3284,TERMINATED,127.0.0.1:29578,relu,23.4338,100,14.8323,2.41102,100,11.5858,99,-2.41102
objective_ea4af5a2,TERMINATED,127.0.0.1:29580,relu,-69.6225,100,4.19493,-6.72712,100,11.5853,99,6.72712


Result for objective_dbda6994:
  date: 2022-10-18_15-21-27
  done: false
  experiment_id: cc508066dd1c4297ab64129c2e37253a
  hostname: Phams-MacBook-Pro-2.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 127.0.0.1
  pid: 29547
  time_since_restore: 0.10042023658752441
  time_this_iter_s: 0.10042023658752441
  time_total_s: 0.10042023658752441
  timestamp: 1666081287
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: dbda6994
  warmup_time: 0.002299070358276367
  
Result for objective_dcb5d6fa:
  date: 2022-10-18_15-21-29
  done: false
  experiment_id: 4aea026276c64f489aa8c29748144025
  hostname: Phams-MacBook-Pro-2.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 127.0.0.1
  pid: 29548
  time_since_restore: 0.10367012023925781
  time_this_iter_s: 0.10367012023925781
  time_total_s: 0.10367012023925781
  timestamp: 1666081289
  timesteps_since_restore: 0
  training

2022-10-18 15:26:35,388	INFO tune.py:758 -- Total run time: 309.80 seconds (308.97 seconds for the tuning loop).
